In [7]:
import Tkinter, tkFileDialog, os
from Libs import LVQLib
from Libs import ExpWriter as ew

In [8]:
class LVQUI(Tkinter.Tk):    
    """
    Views:

        1) Load in samples
        2) Label Classifications -- show the network
        3) Training & save screen

    """
    
    def __init__(self, parent):
        self.viewdex = 0        # current view index
        self.parent  = parent
        self.state   = { 'savePath': None }
        self.views   = ['Load Samples', 'Label Neurons', 'Train Network']
        self.fpaths  = []
        
        Tkinter.Tk.__init__(self, parent)
        self.initialize()
        
    def initialize(self):
        self.grid()
        
        self.grid_columnconfigure(0, weight=2)
        self.resizable(False, True)
        
        # Labels
        self.screen_title = Tkinter.Label(self, text=self.views[self.viewdex], anchor='center')
        self.files_label  = Tkinter.Label(self, text="Files Imported:") 
        
        # Buttons
        self.forwardBtn   = Tkinter.Button(self, text=" Next  >>>", command=self.nextView,
                                           width=10)
        self.backwardBtn  = Tkinter.Button(self, text="<<<  Back ", command=self.prevView,
                                           width=10)
        
        # Grid Positioning
        self.screen_title.grid(column=4, row=1, columnspan=8, sticky='W',  padx=50)
        self.files_label.grid(column=0,  row=2, columnspan=2, sticky='N', pady=100)
        self.forwardBtn.grid(column=16,  row=1, columnspan=4, sticky='E')
        self.backwardBtn.grid(column=0,  row=1, columnspan=4, sticky='W')
        
        
        self.update()
        self.geometry(self.geometry()) # comment out for auto resizing.
            
    def getFileDialogue(self):
        filey = tkFileDialog.askopenfile(parent=self, mode='r', 
                                         title="Audio Sample")
        if filey != None:
            data = filey.read()
            self.state['filepath'] = filey.name
            filey.close()
    
    def nextView(self):
        if (self.viewdex < len(self.views)):
            self.viewdex += 1
            
        print self.viewdex
        self.update()
        
    def prevView(self):
        if (self.viewdex > 0):
            self.viewdex -= 1
        
        print self.viewdex
        self.update()
        
    def loadViewInit(self):
        """
        Load the view (1): 
            Load samples for training
            (and vector initialization of the neurons)
        """
        pass
    
    def labelViewInit(self):
        """
        Load the view (2): 
            Label neuron classifiers
        """
        pass
    
    def trainViewInit(self):
        """
        Load the view (3):
            Training Parameters & Output / Progress
        """
        pass

In [10]:
if __name__ == '__main__':
    gui = LVQUI(None)
    gui.title('LVQ Drum Training')
    gui.mainloop()
    
    # Testing writing expressions
    w = ew.SEWriter('network', './')
    w.finish()